In [1]:
import h5py
import matlab.engine
import pandas as pd
import sys
import os

In [2]:
eng = matlab.engine.start_matlab()

In [3]:
def extract_fit_info(mrs_fit):
    gaba = mrs_fit["GABA"]

    gaba_linewidth = float(gaba["FWHM"])
    gaba_snr = float(gaba["SNR"])
    gaba_fit_error = float(gaba["FitError"])
    gaba_conc_water = float(gaba["ConcIU"])
    gaba_conc_cr = float(gaba["ConcCr"])

    glx = mrs_fit["Glx"]

    glx_linewidth = float(glx["FWHM"])
    glx_snr = float(glx["SNR"])
    glx_fit_error = float(glx["FitError"])
    glx_conc_water = float(glx["ConcIU"])
    glx_conc_cr = float(glx["ConcCr"])

    return {
        "gaba_snr":gaba_snr,
        "gaba_linewidth":gaba_linewidth,
        "gaba_fit_error":gaba_fit_error,
        "gaba_conc_water":gaba_conc_water,
        "gaba_conc_cr":gaba_conc_cr,
        "glx_snr":glx_snr,
        "glx_linewidth":glx_linewidth,
        "glx_fit_error":glx_fit_error,
        "glx_conc_water":glx_conc_water,
        "glx_conc_cr":glx_conc_cr,
    }


In [6]:
## make acc pre_results
input_folder = "../data/gannet_acc_fit_ranges"
filenames = os.listdir(input_folder)
pre_df = []
for filename in filenames:
    row_filename = filename[:6]
    row_range = f"{filename.split('_')[3]}-{filename.split('_')[4]}"
    row_dict = {"filename":row_filename,"range":row_range}
    mrs_fit = eng.load(f"{input_folder}/{filename}")["mrs_fit"]["vox1"]
    row_dict.update(extract_fit_info(mrs_fit))
    pre_df.append(row_dict)
    del mrs_fit

acc_df = pd.DataFrame(pre_df)

acc_df.to_csv("../eval/fit_ranges/acc_ref_df.csv",index=False)    

In [7]:
acc_df.shape

(3600, 12)

In [9]:
ref_df = pd.read_csv("../pre_results/full_ref_df.csv")

In [14]:
i_df = acc_df.copy()

i_df = pd.merge(i_df,ref_df,on="filename",suffixes=("_acc","_ref"))

metric = "gaba_conc_cr"
i_df["abs_diff_mean"] = abs(i_df[f"{metric}_acc"] - i_df[f"{metric}_ref"])
i_df["ref"] = i_df[f"{metric}_ref"]
i_df["mean"] = i_df[f"{metric}_acc"]
i_df["std"] = i_df[f"{metric}_acc"]

a_df = i_df.groupby("filename").agg({"ref":"mean","mean":"mean","std":"std","abs_diff_mean":"mean"})


In [15]:
a_df.sort_values("abs_diff_mean",ascending=False)

,ref,mean,std,abs_diff_mean
filename,,,,
g7_s11,0.187954,-30.902075,114.163166,32.366113
g8_s08,0.839502,1.438407,4.213680,1.316704
p6_s01,0.117541,0.671483,1.932859,0.569358
p4_s10,0.125266,0.589521,0.904422,0.475708
p4_s05,0.284771,0.469445,0.497746,0.350387
...,...,...,...,...
s8_s10,0.109508,0.109793,0.004172,0.003516
p8_s11,0.111600,0.112464,0.004117,0.003486
s1_s06,0.116359,0.114794,0.003805,0.003179
